In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("hdr-data.xlsx")
df

,countryIsoCode,country,indexCode,index,dimension,indicatorCode,indicator,year,value,note
0,AFG,Afghanistan,HDI,Human Development Index,NaN,eys,Expected Years of Schooling (years),2023,10.790143,NaN
1,AFG,Afghanistan,HDI,Human Development Index,NaN,gnipc,Gross National Income Per Capita (2021 PPP$),2023,1971.549458,NaN
2,AFG,Afghanistan,HDI,Human Development Index,NaN,hdi,Human Development Index (value),2023,0.496000,NaN
3,AFG,Afghanistan,HDI,Human Development Index,NaN,hdi_rank,HDI Rank,2023,181.000000,NaN
4,AFG,Afghanistan,HDI,Human Development Index,NaN,le,Life Expectancy at Birth (years),2023,66.035000,NaN
...,...,...,...,...,...,...,...,...,...,...
1157,ZWE,Zimbabwe,HDI,Human Development Index,NaN,hdi_rank,HDI Rank,2023,153.000000,NaN
1158,ZWE,Zimbabwe,HDI,Human Development Index,NaN,gnipc,Gross National Income Per Capita (2021 PPP$),2023,3511.463215,NaN
1159,ZWE,Zimbabwe,HDI,Human Development Index,NaN,eys,Expected Years of Schooling (years),2023,11.057453,NaN
1160,ZWE,Zimbabwe,HDI,Human Development Index,NaN,mys,Mean Years of Schooling (years),2023,8.930847,NaN


In [6]:
df[df['country'] == 'Russia']

,countryIsoCode,country,indexCode,index,dimension,indicatorCode,indicator,year,value,note


In [26]:
# List of countries to filter
final_countries = ["Mexico", "China", "India", "Russia", "Spain", "France", "Portugal",
             "Brazil", "Morocco", "Sweden", "Finland", "Poland", "United Kingdom", 
             "Ireland" , "Libya", "Sudan", "Saudi Arabia", "Iraq", "Thailand", "South Africa", 
             "Indonesia"]

In [15]:
#df = df.drop(columns=["dimension", "note"])

In [27]:
# Filter only the HDI indicator
df_hdi = df[df["indicatorCode"] == "hdi"].copy()

df_hdi = df_hdi[["countryIsoCode", "country", "year", "value"]]

df_hdi.rename(columns={"value": "HDI"}, inplace=True)

In [28]:
#STEP 3. Calculate HDI ranking for ALL countries
# -------------------------------
df_hdi_sorted = df_hdi.sort_values(by="HDI", ascending=False).reset_index(drop=True)
df_hdi_sorted['HDI_Rank'] = df_hdi_sorted['HDI'].rank(ascending=False, method='dense')

# Save the complete global ranking
df_hdi_sorted.to_csv("hdi_ranking_all_countries.csv", index=False)


In [29]:
# STEP 4. Filter only your final countries
# -------------------------------
df_final = df_hdi_sorted[df_hdi_sorted['country'].isin(final_countries)].copy()


In [30]:
import pycountry_convert as pc

# Function to convert ISO code to continent
def country_to_continent(country_code):
    try:
        country_alpha2 = pc.country_alpha3_to_country_alpha2(country_code)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = {
            "AF": "Africa",
            "AS": "Asia",
            "EU": "Europe",
            "NA": "North America",
            "SA": "South America",
            "OC": "Oceania"
        }
        return continent_name[continent_code]
    except:
        return None

# Create column with continent
df_final["continent"] = df_final["countryIsoCode"].apply(country_to_continent)

df_final.head()

,countryIsoCode,country,year,HDI,HDI_Rank,continent
5,SWE,Sweden,2023,0.959,4.0,Europe
10,IRL,Ireland,2023,0.949,8.0,Europe
11,FIN,Finland,2023,0.948,9.0,Europe
12,GBR,United Kingdom,2023,0.946,10.0,Europe
25,FRA,France,2023,0.920,20.0,Europe


In [31]:
df_final

,countryIsoCode,country,year,HDI,HDI_Rank,continent
5,SWE,Sweden,2023,0.959,4.0,Europe
10,IRL,Ireland,2023,0.949,8.0,Europe
11,FIN,Finland,2023,0.948,9.0,Europe
12,GBR,United Kingdom,2023,0.946,10.0,Europe
25,FRA,France,2023,0.920,20.0,Europe
27,ESP,Spain,2023,0.918,22.0,Europe
34,POL,Poland,2023,0.906,26.0,Europe
36,SAU,Saudi Arabia,2023,0.900,28.0,Asia
39,PRT,Portugal,2023,0.890,31.0,Europe
76,THA,Thailand,2023,0.798,61.0,Asia


In [35]:
df_final.rename(columns={"HDI_Rank": "category"}, inplace=True)

In [36]:
# STEP 7. Select only the final columns
# -------------------------------
columns_to_keep = ['countryIsoCode', 'country', 'HDI', 'year', 'continent', 'category']
df_final = df_final[columns_to_keep]

In [37]:
df_final

,countryIsoCode,country,HDI,year,continent,category
5,SWE,Sweden,0.959,2023,Europe,4.0
10,IRL,Ireland,0.949,2023,Europe,8.0
11,FIN,Finland,0.948,2023,Europe,9.0
12,GBR,United Kingdom,0.946,2023,Europe,10.0
25,FRA,France,0.920,2023,Europe,20.0
27,ESP,Spain,0.918,2023,Europe,22.0
34,POL,Poland,0.906,2023,Europe,26.0
36,SAU,Saudi Arabia,0.900,2023,Asia,28.0
39,PRT,Portugal,0.890,2023,Europe,31.0
76,THA,Thailand,0.798,2023,Asia,61.0


In [38]:
# Save the complete global ranking
df_hdi_sorted.to_csv("hdi_ranking_all_countries.csv", index=False)

In [39]:
# STEP 8. Save the filtered countries
df_final.to_csv("hdi_ranking_filtered_countries.csv", index=False)